# Projektvorstellung Natural Language Processing

&copy; Daniel Schaudt, [Prof. Dr. Reinhold von Schwerin](https://www.thu.de/Reinhold.vonSchwerin), Technische Hochschule Ulm

In diesem Notebook soll aufgezeigt werden, wie mit Deep Learning-Methoden ein einfacher Chatbot erstellt werden kann, welcher Fragen zu bestimmten Themen beantwortet. Dieser Bereich des Machine Learning lässt sich dem *Natural Language Processing (NLP)* zuordnen - also der Verarbeitung von Sprache, die von Menschen benutzt wird. Dieses Notebook orientiert sich an einem Blogpost von [Dirk Hornung](https://chatbotslife.com/how-to-build-a-chatbot-from-zero-a0ebb186b070).

Der verwendete Technologie-Stack besteht im Kern aus `Python` und der Bibliothek `Tensorflow`. Alle weiteren Bibliotheken dienen als Helfer für die Datenverarbeitung. Entgegen der vorherrschenden Meinung, dass für Deep Learning besonders viele Daten benötigt werden, kommt diese Demonstration mit nur wenigen Daten aus (4 KB Textfile). Dadurch soll gezeigt werden, dass auch das eigenständige Sammeln und *labeln* von Daten in einem Deep Learning-Projekt zum Erfolg führen kann.

**Anmerkung:** Deep Learning ist ein komplexes Thema! Das Ziel dieses Notebooks ist es, anhand eines einfachen Anwendungsfalles einen groben Überblick über Deep Learning-Methoden zu geben, welcher von Lesern mit unterschiedlichem Hintergrund verstanden werden kann. Für interessierte Leser stellt dieses Notebook aber immer wieder Literaturreferenzen bereit, welche das Thema detaillierter aufarbeiten.

Um Datenprojekte strukturiert aufzubauen, bietet sich der Einsatz eines etablierten Vorgehensmodells an. Diese Projektvorstellung orientiert sich am [CRISP-DM-Modell](https://en.wikipedia.org/wiki/Cross-industry_standard_process_for_data_mining). Dieses Modell deckt die folgenden Projekt-Phasen ab:

![CRISP-DM](./images/Cross-Industry-Standard-Process-for-Data-Mining-CRISP-DM-12_s.png "CRISP-DM")


Um den Aufbau verständlich zu halten, entsprechen die Überschriften dieses Notebooks genau den Phasen des CRISP-DM-Prozesses. Die Kurzbeschreibung der Phasen in diesem Notebook entstammen aus [Pete Chapman et al](ftp://ftp.software.ibm.com/software/analytics/spss/support/Modeler/Documentation/14/UserManual/CRISP-DM.pdf).

## Business Understanding

> **Aus CRISP-DM 1.0:** *Diese erste Phase konzentriert sich auf das Verständnis der Projektziele und -anforderungen aus der Geschäftsperspektive. Dieses Wissen wird dann in eine Data-Mining-Problemdefinition und einen vorläufigen Plan umgewandelt, um
die Ziele zu erreichen.*

Laut [IBM 2017](https://www.ibm.com/blogs/watson/2017/10/how-chatbots-reduce-customer-service-costs-by-30-percent/) geben Unternehmen jedes 1,3 Billionen Dollar für Kundenservice-Anrufe aus. Wenn Chatbots in der Lage sind auch nur einen geringen Teil der 265 Milliarden Anrufe obsolet zu machen, dann sind gigantische Einsparungen zu erwarten. Der in diesem Notebook vorgestellte, *sehr einfache* Chatbot könnte wahrscheinlich nur einen Teil des First-Level-Supports übernehmen - diese Routineanfragen machen laut IBM aber 80% aller Anfragen aus.

Konkret soll hier ein Chatbot für Anfragen an ein *medizinisches Enterprise-System* vorgestellt werden. Der Chatbot soll vor allem neue Nutzer schnell mit Informationen versorgen und (in der implementierten Variante) auch einfache API-Aufrufe absetzen können.

## Data Understanding

> **Aus CRISP-DM 1.0:** *Die Phase Data Understanding beginnt mit der ersten Datensammlung und geht weiter mit Aktivitäten, die es ermöglichen, sich mit den Daten vertraut machen. Dazu gehören: Datenqualitätsprobleme zu identifizieren, erste Einblicke in die Daten zu gewinnen und/oder interessante Teilmengen zu entdecken, um Hypothesen über verborgene Informationen zu bilden.*

### Reproduzierbarkeit erreichen

Setzen diverser [Random Seeds](https://de.wikipedia.org/wiki/Seed_key) für determistisches Trainingsverhalten (d.h. zur Reproduzierbarkeit der Ergebnisse):

In [1]:
RSEED = 42
import random as rand
rand.seed(RSEED)
from numpy.random import seed
seed(RSEED)
from tensorflow import random
random.set_seed(RSEED)

**Bemerkung** Leider erreicht man damit u.U. nur eine *lokale Reproduzierbarkeit*, d.h. die Ergebnisse auf einem bestimmten Rechner bleiben bei auch sonst unveränderten Modellparametern (s.u.) gleich. Rechnerübergreifend kann aber zum Tragen kommen, was [Jason Brownlee](https://machinelearningmastery.com/reproducible-results-neural-networks-keras/) im Abschnitt *Randomness from a Sophisticated Model* schreibt. Ohnehin sollte man Brownlees Empfehlung folgen und zur Beurteilung der Performance des Modells dieses mehrfach ausführen.

Im Falle des doch recht einfachen Modells, welches hier generiert wird, ist aber offenbar die Reproduzierbarkeit über verschiedene Rechner hinweg gegeben.

### Import der Bibliotheken

Nachfolgend werden die benötigten Bibliotheken importiert. Eine kurze Information zu den wichtigsten Bibliotheken:
* [Tensorflow](https://www.tensorflow.org/): Open-source Machine Learning-Bibliothek, welche den Kern dieses Notebooks bildet. Mit Tensorflow werden die Deep Learning-Modelle erzeugt und trainiert. Genauer gesagt wird in diesem Notebook die High-Level API [Keras](https://www.tensorflow.org/guide/keras/overview) verwendet, welche seit Tensorflow 2.0 fest integriert ist. Auch für die Vorverarbeitung der Daten wird die Bibliothek benutzt.
* [numpy](https://numpy.org/): Eine mächtige Bibliothek für wissenschaftliche Berechnungen. Wir verwenden in diesem Notebook die Array-Funktionalitäten von numpy.
* [pandas](https://pandas.pydata.org/): Mächtiges Werkzeug zur Verarbeitung tabellarischer Daten in Python. Wird hier lediglich für die optisch ansprechende Ausgabe der Testergebnisse verwendet.

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
import json
import numpy as np
import sys
import pandas as pd

### Grundlegendes zum Modell

Ein einfacher, wie in diesem Notebook vorgestellter *Contextual Chatbot* versucht die Intention einer Anfrage zu verstehen. Im Grunde versucht der Chatbot also ein *Mapping* eines *Inputs* auf eine *Absicht (Intent)* zu erlernen: 

![Chatbot Basics](./images/chatbot.png "Chatbot Basics")

> Bild von [Dirk Hornung](https://chatbotslife.com/how-to-build-a-chatbot-from-zero-a0ebb186b070)

Wenn die Absicht erkannt wurde, dann soll eine vordefinierte Antwort ausgelöst werden. Es wäre aber auch denkbar externe Datenquellen als Antwort anzuzapfen. Mit diesem Wissen ist der Aufbau der Trainingsdaten besser zu verstehen.

### Laden der Textdaten

> Die Textdaten stammen aus Dataflairs [Chatbot Projekt](https://data-flair.training/blogs/python-chatbot-project/), wobei die Daten ursprünglich vermutlich von [Andrej Baranovskij](https://andrejusb.blogspot.com/2018/03/classification-machine-learning-chatbot.html) übernommen wurden.

Die Daten liegen im `json`-Format vor und befinden sich im Ordner `./data`. Mit `json.load()` können diese wie folgt geladen werden: 

In [3]:
with open('./data/intents_health.json', 'r') as f:
    intents = json.load(f)

Das resultierende Objekt ist ein Python [Dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) mit folgendem Inhalt:

In [4]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do?',
    'What help you provide?',
    'How you can be helpful?',
    'What suppo

Die Datei enthält folgende Elemente:
* **Tag:** Entspricht dem *Label* oder *Intent* einer Nachricht, also der Absicht einer Anfrage.
* **Patterns:** Beispielhafte *Anfragemuster*, welche möglicherweise im Zusammenhang mit einem *Tag* auftreten.
* **Responses:** Mindestens eine Antwortmöglichkeiten für den Chatbot.

Weiterhin ist anzumerken, dass die Datei wirklich nicht groß ist. Sie enthält lediglich 10 Tags und nur wenige Patterns. Die Anzahl der Patterns pro Tag sollten zudem ausgewogen sein, da das Modell sonst dazu neigt, jene Tags mit den meisten Patterns auszuwählen.

## Data Preparation

> **Aus CRISP-DM 1.0:** *Die Phase Data Preparation umfasst alle Aktivitäten, die zur Erstellung des finalen Datensatzes aus den anfänglichen Rohdaten benötigt werden. Datenvorbereitungsaufgaben werden unter Umständen mehrfach und nicht
in vorgegebener Reihenfolge ausgeführt. Zu den Aufgaben gehören die Auswahl von Tabellen, Datensätzen und Attributen sowie die Transformation und Bereinigung von Daten für die Modellierung.*

Der Input eines Machine Learning-Modells muss numerisch sein - mit den reinen Wörtern (strings) kann das Modell leider nicht arbeiten. Die Sätze müssen also als eine Sequenz von Zahlen dargestellt werden. Dieser Prozess, bei welchem jedem Wort eine einzigartige ID zugeordnet wird, nennt sich *Tokenization* und ist im Grunde nichts anderes als ein Mapping. Zu diesem Zweck wird das Vokabular der Patterns zunächst in eine Liste `intent_list` umgewandelt:

In [5]:
intent_list = []
label_list = []
for index, intent in enumerate(intents['intents']):
    intent_list += intent['patterns']
    num_patterns = len(intent['patterns'])
    label_list += [index] * num_patterns

Ebenso wird eine `label_list` erzeugt, welche jedem Pattern den zugehörigen Tag zuordnet. Diese wird später für den Trainingsprozess benötigt, damit das Modell die korrekte Zuordnung der Patterns zu den Tags erlernt.

Anschließend findet der Tokenization-Prozess statt. Es wird der Tokenizer aus `keras.preprocessing` verwendet und auf die `intent_list` angepasst:

In [6]:
oov_tok = '<OOV>' #Out-of-Vocabulary Token
tokenizer = Tokenizer(oov_token = oov_tok)
tokenizer.fit_on_texts(intent_list)
word_index = tokenizer.word_index

Das `oov_tok` entspricht einem *Out-of-Vocabulary-Token*, also einem Platzhalter für unbekannte Wörter, die in dem *Textkorpus* der Trainingsdaten nicht vorkommen. Das Modell soll schließlich auch alle Eingaben erkennen, welche *ähnlich* zu den festgelegten Patterns sind. Würde es lediglich die genauen Patterns erkennen, dann bräuchten wir kein Machine Learning, sondern eine Tabelle!

In `word_index` sind nun alle Wörter mit einer ID als Key-Value-Paar in einem [Python-Dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) gespeichert. Die ersten 5 Wörter sind:

In [7]:
dict(list(word_index.items())[0:5])

{'<OOV>': 1, 'blood': 2, 'pressure': 3, 'you': 4, 'for': 5}

Nun können die einzelnen Patterns in Zahlenfolgen umgewandelt werden. Dies geschieht bequem mit der `texts_to_sequences`-Funktion eines angepassten tokenizer-Objekts. Beispielhaft hier die ersten 10 Sätze:

In [8]:
sequences = tokenizer.texts_to_sequences(intent_list); sequences[:10]

[[36, 24],
 [10, 37, 4],
 [25, 38, 24],
 [39],
 [40],
 [41],
 [42, 43],
 [26],
 [44, 4, 45],
 [46]]

Der Input des hier gezeigten neuronalen Netzes muss immer die gleich Länge aufweisen. Da die Sätze aber unterschiedlich lang sind, müssen diese *aufgefüllt* werden. Dieser Prozess wird als *Padding* bezeichnet. Konkret wird nach dem längsten Satz gesucht und die restlichen Sätze dann auf die gleiche Länge mit Nullen aufgefüllt. Nähere Informationen zu Padding gibt es u.a. in diesem [Blogpost](https://machinelearningmastery.com/data-preparation-variable-length-input-sequences-sequence-prediction/).

In [9]:
padded = pad_sequences(sequences)
padded_length = len(padded[0])

Hier sieht man das Padding in einem Ausschnitt, welcher die längsten Sätze enthält:

In [10]:
padded[20:30]

array([[ 0,  0,  0,  0, 10,  4, 29, 59, 27],
       [ 0,  0,  0,  0,  0, 15, 60, 25, 61],
       [ 0,  0,  0, 10,  6, 62,  8, 63, 16],
       [ 0,  0,  0,  0,  0, 30,  8, 12, 31],
       [64, 11, 32, 17, 33, 12, 65,  8, 66],
       [17, 67, 12, 68,  5,  7, 69,  8, 16],
       [ 0,  0,  0, 70, 12, 71, 72,  8, 16],
       [ 0,  0,  0,  0,  0, 30,  2,  3, 31],
       [ 0,  0,  0,  0, 73, 74,  6,  2,  3],
       [ 0,  0,  0,  0,  0,  2,  3, 18, 75]], dtype=int32)

Die Größe des Vokabulars wird noch definiert, da diese für das Deep Learning-Modell später wichtig ist:

In [11]:
vocab_size = len(word_index)
vocab_size

90

In der aktuellen Struktur der Daten ist jedem Pattern ein Tag zugeordnet, welches mit einer Zahl (0-9 für 10 verschiedene Tags) kodiert ist. Damit das Deep Learning-Modell später aber eine Vorhersage treffen kann, eignet sich eine Darstellung der Labels als *One-Hot-Encoding*. Jedes Label wird dadurch zu einem binären *On-Off-Switch*. Mehr Informationen zu One-Hot-Encoding gibt es u.a. auf [Hackernoon](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). Am einfachsten kann man sich das an den Daten verdeutlichen:

In [12]:
labels = tf.keras.utils.to_categorical(label_list); labels[:10]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

Die Länge der Labels entspricht nun also der Anzahl an Tags. Die Zugehörigkeit zu einem Tag wird mit einer 1 (*Hot*) repräsentiert.

Abschließend wird noch die Anzahl an Kategorien definiert, da diese für die Größe des Outputs des Deep Learning-Modells relevant sind:

In [13]:
num_categories = len(labels[0])

## Modeling

> **Aus CRISP-DM 1.0:** *In dieser Phase werden verschiedene Modellierungstechniken ausgewählt und angewendet und ihre Parameter auf optimale Werte kalibriert. Normalerweise gibt es mehrere Techniken für denselben Data-Mining-Problemtyp. Einige Techniken haben spezifische Anforderungen an die Form der Daten. Daher ist es oft notwendig, zur Datenvorbereitungsphase zurückzugehen.*

In diesem Abschnitt wird das Deep Learning-Modell definiert und trainiert. Für NLP-Aufgaben können eine Reihe von verschiedenartigen neuronalen Netzen zum Einsatz kommen. Im Rahmen dieser Projektvorstellung wird ein einfaches *Feedforward-Netzwerk* verwendet. Für komplexere Aufgaben werden jedoch häufig *Recurrent Neural Networks (RNN)* oder *Transformer Networks* eingesetzt.

Weitere Informationen zu neuronalen Netzen im Allgemeinen und modernen NLP-Varianten im Speziellen finden sich u.a. in folgender Literatur:
* Exzellente Videoeinführung zu NNs von 3Blue1Brown, unbedingt [anschauen!](https://www.youtube.com/watch?v=aircAruvnKk&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi)
* Grundlegendes Konzept von neuronalen Netzen mit interaktiven Beispielen und Visualisierungen: [Jay Alammar](https://jalammar.github.io/visual-interactive-guide-basics-neural-networks/)
* Ebenfalls auf [Jay Alammar's Blog](http://jalammar.github.io/) finden sich gelungene Visualisierungen zu aktuellen NLP-Architekturen wie BERT, GPT-2 und weiteren Seq2Seq-Modellen.
* Interessantes Video zu Googles aktueller Forschung im Bereich Chatbots: [Two Minute Papers](https://www.youtube.com/watch?v=3Wppf_CNvD0)

Die einzelnen Ebenen (*Layer*) des Modells werden nachfolgend definiert:

In [14]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size+1, 16, input_length=padded_length),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_categories, activation='softmax')
])

Das Modell hat gegenüber einem einfachen Feedforward-Netzwerk allerdings eine Besonderheit: [Word Embeddings](https://machinelearningmastery.com/what-are-word-embeddings/)! Durch diese wird jedes Wort in einen n-dimensionalen Vektorraum projiziert. Das bedeutet ein Wort wird nicht nur durch *eine* Zahl repräsentiert, sondern durch mehrere. In diesem Fall werden 16 Dimensionen verwendet. Dies hat im Kern zwei Vorteile: 

1. Ein Wort kann durch mehrere Features repräsentiert werden. Als Mensch könnte man sich folgende Features vorstellen: Gefühlslage, Sprachniveau, Alter des Wortes, etc. Welche "Features" das Modell nun genau erkennt, kann leider nicht nachvollzogen werden.
2. Die Platzierung des Wortes in einem Vektorraum ermöglicht die Berechnung mit *Distanzen*. So können *ähnliche* Wörter dicht zusammen liegen und unähnliche weit voneinander entfernt. Eine interaktive 3D Visualisierung eines solchen Wort-Vektorraumes gibt es z.B. [hier](http://projector.tensorflow.org/).

Nachfolgend werden Trainingsparameter für das Modell bestimmt. Das Training entspricht einer Anpassung der Gewichte des Netzwerks mithilfe des stochastischen Gradientenverfahrens. Umfassende Erklärungen des Verfahrens sprengen leider den Rahmen dieser Einführung. Für weitere Informationen zum Training eines neuronalen Netzes wird daher auf [Nielsen](http://neuralnetworksanddeeplearning.com/chap2.html) verwiesen.

In [15]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc'])

Eine Zusammenfassung des Modells erhält man mit `summary()`:

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 9, 16)             1456      
_________________________________________________________________
flatten (Flatten)            (None, 144)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               18560     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                6

Die Zusammenfassung zeigt die Architektur des Netzwerks, also die Layer sowie deren Art von Beginn des Netzwerks (Inputlayer) bis zum Ende (Outputlayer). Weiterhin kann die Größe der Layer und deren Verbindungen untersucht werden. Folgende Parameter des Modells werden noch definiert:

* **Batchsize:** Die Batchsize gibt an, wieviele Sätze in einem *Durchgang* durch das neuronale Netz geleitet werden. Dieser Parameter ist im Wesentlichen aus zwei Gründen wichtig: 1) zur Speicheroptimierung und 2) für das Training des Modells. Die Wahl der Batchsize kann ein heikles Thema darstellen und hängt von vielen Faktoren ab. Eine Einführung geben u.a. [Michael Nielsen](http://neuralnetworksanddeeplearning.com/chap2.html) oder Kapitel 8.1.3 von [Ian Goodfellow et al.](https://www.deeplearningbook.org/contents/optimization.html).
* **Epochs:** Eine Epoch entspricht einem kompletten Durchlauf aller Trainingsdaten 

In [17]:
BATCH_SIZE = 5
EPOCHS = 25

Abschließend folgt nun das Training des Modells:

In [18]:
history = model.fit(padded, labels, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2)

Train on 47 samples
Epoch 1/25
47/47 - 1s - loss: 2.3018 - acc: 0.0638
Epoch 2/25
47/47 - 0s - loss: 2.2842 - acc: 0.1702
Epoch 3/25
47/47 - 0s - loss: 2.2599 - acc: 0.3830
Epoch 4/25
47/47 - 0s - loss: 2.2381 - acc: 0.3404
Epoch 5/25
47/47 - 0s - loss: 2.2160 - acc: 0.3830
Epoch 6/25
47/47 - 0s - loss: 2.1628 - acc: 0.4468
Epoch 7/25
47/47 - 0s - loss: 2.0792 - acc: 0.5319
Epoch 8/25
47/47 - 0s - loss: 2.0055 - acc: 0.5106
Epoch 9/25
47/47 - 0s - loss: 1.8513 - acc: 0.5106
Epoch 10/25
47/47 - 0s - loss: 1.7100 - acc: 0.5745
Epoch 11/25
47/47 - 0s - loss: 1.5415 - acc: 0.5532
Epoch 12/25
47/47 - 0s - loss: 1.3260 - acc: 0.7021
Epoch 13/25
47/47 - 0s - loss: 1.2000 - acc: 0.6596
Epoch 14/25
47/47 - 0s - loss: 1.0661 - acc: 0.7021
Epoch 15/25
47/47 - 0s - loss: 0.9734 - acc: 0.7447
Epoch 16/25
47/47 - 0s - loss: 0.8236 - acc: 0.8298
Epoch 17/25
47/47 - 0s - loss: 0.7038 - acc: 0.8511
Epoch 18/25
47/47 - 0s - loss: 0.5729 - acc: 0.8723
Epoch 19/25
47/47 - 0s - loss: 0.6210 - acc: 0.8085
E

## Evaluation

> **Aus CRISP-DM 1.0:** *In dieser Phase des Projekts wurde ein Modell erstellt, welches eine ausreichende Performance zu haben scheint. Bevor das Modell endgültig implementiert wird, ist es wichtig, das Modell, sowie die Schritte, die zur Erstellung des Modells geführt haben, gründlich zu evaluieren und zu überprüfen, um sicher zu sein, dass das Modell die Geschäftsziele ordnungsgemäß erreicht. Es soll sichergestellt werden, dass alle Geschäftsziele ausreichend berücksichtigt wurden. Am Ende dieser Phase sollte eine Entscheidung über die Verwendung der Data-Mining-Ergebnisse getroffen werden.*

In diesem Abschnitt wird die Performance des Modells untersucht. **Es soll betont werden, dass es sich hierbei um eine *empirische Untersuchung* handelt** - es werden also Testsätze *ausprobiert* um ein Gefühl für die Performance des Modells zu bekommen. Diese Art der Evaluation ist der Datenknappheit geschuldet! In der Realität sollte das Modell anhand von Testdaten untersucht werden und eine statistische Auswertung von etablierten Metriken erfolgen. Die Testdaten sind derart zu wählen, dass das Modell diese **nicht** während des Trainingslaufs schon gesehen hat - sonst kann keine valide Beurteilung vorgenommen werden.

Wie oben aus der Trainingshistorie ausgelesen werden kann, hat das Modell nach 25 Epochen (Trainingsdurchläufen) eine Genauigkeit von 97,87% erzielt. Das bedeutet, das Modell ordnet jedem Pattern in den Trainingsdaten zu 97,87% den zugehörigen *Tag* korrekt zu. Wie schon angemerkt, hat diese Angabe aber nur einen begrenzten Nutzen für die Anwendung in der Praxis! Es könnte ja sein, dass das Modell nur die Trainingsdaten *auswendig* lernt und nicht in der Lage ist auf Abweichungen entsprechend zu reagieren. Das soll nun mit einigen Beispielsätzen ausprobiert werden.

Zu diesem Zweck wird nachfolgend eine Funktion definiert, welche zu einem Input-Satz den vorhergesagten Tag zurückgibt:

In [19]:
def predict_tag(sentence):
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=padded_length)
    prediction = model.predict(padded_sequence)[0]
    intent = intents['intents'][np.argmax(prediction)]
    tag = intent['tag']
    return tag

Zur Erinnerung, folgende Tags sind im Datensatz vorhanden:

In [20]:
tag_list = []
for tags in intents['intents']:
    tag_list.append(tags['tag'])
    print(tags['tag'])

greeting
goodbye
thanks
noanswer
options
adverse_drug
blood_pressure
blood_pressure_search
pharmacy_search
hospital_search


Es wird zu jedem Tag ein Beispielsatz entworfen, welcher **nicht** in dieser Form in den Trainingsdaten vorliegt:

In [21]:
test_sentences = ['Greetings',
                 'It was nice meeting you',
                 'Thank you very much',
                 '',
                 'How can you support me?',
                 'What adverse reaction has this drug?',
                 'I want to enter blood pressure',
                 'What is the blood pressure for patient?',
                 'Where is the neares pharmacy?',
                 'I need data from another hospital']

Die Testsätze können natürlich beliebig einfach (ähnlich) oder schwer (unähnlich) gewählt werden. Dies macht die Beurteilung des Modells nicht leichter. In der Praxis werden Chatbots daher oft im laufenden Betrieb stetig verbessert und dazu Feedback der Benutzer gesammelt ("Hat Ihnen diese Antwort geholfen?").

Die Sätze werden zusammen mit dem tatsächlichen Tag und der Vorhersage in einem `pandas`-DataFrame dargestellt. Das DataFrame dient nur der sauberen Darstellung als Tabelle im Notebook:

In [22]:
data_list = list(zip(test_sentences, tag_list, list(map(predict_tag, test_sentences))))
pd.DataFrame(data_list, columns=['Sentence', 'True Tag', 'Predicted Tag'])

,Sentence,True Tag,Predicted Tag
0,Greetings,greeting,greeting
1,It was nice meeting you,goodbye,blood_pressure
2,Thank you very much,thanks,options
3,,noanswer,greeting
4,How can you support me?,options,options
5,What adverse reaction has this drug?,adverse_drug,adverse_drug
6,I want to enter blood pressure,blood_pressure,blood_pressure
7,What is the blood pressure for patient?,blood_pressure_search,blood_pressure_search
8,Where is the neares pharmacy?,pharmacy_search,pharmacy_search
9,I need data from another hospital,hospital_search,hospital_search


Von den 10 unbekannten Sätzen hat das Modell also 7 richtig klassifiziert. Diese Testgenauigkeit (70%) liegt unter der Trainingsgenauigkeit. Im ML-Kontext spricht man hier von *Overfitting*. Das Modell hat eine Überanpassung an die Trainingsdaten vollzogen und verliert in Folge die Fähigkeit eine generalisierte Aussage für unbekannte Daten zu treffen. In der Praxis muss dieses Phänomen mit weiteren Tests genau unter die Lupe genommen werden. Weitere Informationen zur *Kapazität* eines Deep Learning-Modells finden sich Kapitel 5.2 von [Ian Goodfellow et al.](https://www.deeplearningbook.org/contents/ml.html). 

Es sei gesagt, dass sich das hier gezeigte Modell durch den Einsatz von größeren Datenmengen sehr stark verbessern würde. Dies hat vorwiegend zwei Gründe:
* Deep Learning-Modelle skalieren linear mit mehr Daten (gute Intuition von [Kilian Weinberger](https://youtu.be/kPXxbmBsFxs?t=579))
* Mehr Daten haben eine Vergrößerung des Vokabulars zur Folge. Dies reduziert die Verwendung des Out-of-Vocabulary-Tokens in vorher unbekannten Daten und erhöht die Genauigkeit

Bei der Größe der vorliegenden Daten wäre zu prüfen, ob sich klassische ML-Ansätze nicht sogar besser eignen würden. Dazu zählen unter anderem das [Naive Bayes-Modell](https://sebastianraschka.com/Articles/2014_naive_bayes_1.html). Im Rahmen dieser Projektvorstellung sollte jedoch verdeutlicht werden, dass Deep Learning-Modelle (in begrenztem Rahmen) schon mit wenigen Daten sinnvolle Ergebnisse erzielen können. Dies trifft sogar noch mehr zu, wenn moderne NLP-Ansätze mit *vortrainierten* neuronalen Netzen eingesetzt werden (wie beispielsweise in diesem [Projekt](https://medium.com/huggingface/how-to-build-a-state-of-the-art-conversational-ai-with-transfer-learning-2d818ac26313)).

## Deployment

> **Aus CRISP-DM 1.0:** *Die Erstellung des Modells bedeutet im Allgemeinen nicht das Ende des Projekts. Auch wenn der Zweck des Modells darin besteht, das Wissen über die Daten zu erweitern, müssen die gewonnenen Erkenntnisse so organisiert und präsentiert werden, dass der Kunde sie nutzen kann. Dies beinhaltet oft die Anwendung von "Live"-Modellen innerhalb der Entscheidungsfindungsprozesse einer Organisation - zum Beispiel in der Echtzeit-Personalisierung von Webseiten oder der wiederholten Bewertung von Marketingdatenbanken. Je nach den Anforderungen kann die Deployment Phase so einfach sein wie die Erstellung eines Berichts oder so komplex wie die Implementierung eines wiederholbaren Data Mining-Prozesses im gesamten Unternehmen. In vielen Fällen ist es der Kunde, der die Bereitstellung durchführt und nicht der Data Scientist. Aber selbst wenn der Data Scientist die Bereitstellung durchführt, ist es für den Kunden wichtig zu verstehen, welche Aktionen durchgeführt werden müssen, um die erstellten Modelle tatsächlich zu nutzen*

In dieser Phase soll das Modell (der Chatbot) in die Anwendung gebracht, also *ausgerollt* werden. Für dieses Beispielprojekt könnte man sich die Integration in das Enterprise-System eines Krankhauses vorstellen, in welchem der Bot ebenfalls API-Aufrufe absetzen kann und Mitarbeiter durchs Programm begleitet. Der Bot könnte auch per API (z.B. REST) zugänglich gemacht werden oder als Webservice laufen. Die Umsetzung dieser Use Cases sprengt allerdings den Rahmen dieser Projektvorstellung und bleibt den Teilnehmern überlassen. Es sei jedoch gesagt, dass alle Tensorflow-Modelle gespeichert, exportiert und portiert werden können. Nähere Informationen findet man unter [Tensorflow TFX](https://www.tensorflow.org/tfx).

Ein wenig "Deployment" gibt es dann doch: der Chatbot kann innerhalb dieses Notebooks selbst ausprobiert werden. Der Bot wählt dabei zufällig eine Antwortmöglichkeit aus den Responses aus.

In [23]:
def chat():
    print("Start talking with the bot (type \"quit\" to stop)!")
    print('\nBot: Hi! What can I do for you?')
    while True:
        inp = input("Sie: ")
        if inp.lower() == "quit":
            break
            
        sequence = tokenizer.texts_to_sequences([inp])
        padded_sequence = pad_sequences(sequence, maxlen=padded_length)
        prediction = model.predict(padded_sequence)[0]
        pred_class_coded = np.argmax(prediction)
        pred_class_explicit = tag_list[pred_class_coded]
        intent = intents['intents'][pred_class_coded]
        random_answer = rand.choice(intent['responses'])
        
        print(f"Bot: To your {pred_class_explicit} I say {random_answer}")

Zum Ausprobieren einfach folgenden Funktionsaufruf auskommentieren. Viel Spass!

In [24]:
# chat()

## Wie geht es weiter?

### Mögliche Verbesserungen

Das hier vorgestellte Deep Learning-Modell soll als Einstieg in die Thematik dienen und ist daher eher als ein *Grundgerüst* zu verstehen, welches noch an einigen Stellen verbessert werden kann. Diese Verbesserungen obliegen den Teilnehmern, wobei in diesem Abschnitt diverse Verbesserungsansätze (ohne Anspruch auf Vollständigkeit) aufgezeigt werden sollen:

* **Anpassung von Hyperparametern:** Die Hyperparameter sind jene Parameter, die das Netzwerk nicht selbst während des Trainings anpasst (*lernt*), sondern die vom Anwender vor Trainingsbeginn gesetzt werden. Dazu gehört beispielsweise die *Lernrate*, die *Batchsize*, die Anzahl der *Epochen*, die Art des *Optimierers* und die Architektur des Netzwerks selbst, sowie viele Weitere. Die Werte dieser Parameter haben einen monumentalen Effekt auf das Training des neuronalen Netzes und bilden oft ein instabiles Konstrukt, wodurch kleine Änderungen oft schon große Auswirkungen auf die Effektivität des resultierenden Netwerks haben können. Die Einstellung der Hyperparameter erfordert daher zumindest ein grundlegendes Verständnis über deren Wirkungsweise. Die Anpassung kann dann in einem *Trial-and-Error*-Verfahren erfolgen. Eine elegantere Herangehensweise ist die Verwendung von (stochastischen) Suchalgorithmen, welche in einem gegebenen Hyperparameterraum das beste Modell finden. Eine einfache Implementierung solcher Algorithmen findet sich u.a. in der `keras-tuner`-[Bibliothek](https://keras-team.github.io/keras-tuner/).
* **Verwendung eines anderen Modells:** Das in diesem Notebook gezeigte Deep Learning-Modell ist als Feedforward-Modell denkbar einfach und entspricht leider nicht mehr dem aktuellen Stand der Wissenschaft (und Praxis!) im NLP-Bereich. Für eine Textklassifikationsaufgabe würde zumindest ein *Recurrent Neural Network* zum Einsatz kommen, welches durch seine spezielle Modellarchitektur in der Lage ist, eine Sequenz von Ereignissen zu modellieren. Auf die Aufgabenstellung bezogen bedeutet das, ein RNN könnte erkennen, in welcher Reihenfolge die Worte angeordnet sind und daher eine feinere Klassifikation erzielen. Für eine Einführung in RNNs und warum diese so effektiv sind, wird dem geneigten Leser unbedingt ein fantastischer Blogpost von [Andrej Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) ans Herz gelegt. Für Hinweise über eine Umsetzung in Tensorflow ist dieses [Tutorial](https://www.tensorflow.org/tutorials/text/text_classification_rnn) hilfreich.
* **Verwendung vortrainierter Word-Embeddings:** Die Word-Embeddings unseres Modells wurden mitsamt des Modells trainiert und erlernen die Bedeutung von Wörtern anhand der Trainingsdaten. Gerade weil diese Datenmenge aber stark begrenzt ist, könnte es sinnvoll sein, bereits trainierte Word-Embeddings zu verwenden. Diese wurden an extrem umfassenden Textkorpussen erprobt (mehrere GB an Textdaten). Dies hat den Vorteil, dass mit Wortähnlichkeiten gearbeitet werden kann, welche in den Trainingsdaten nicht vorkommen. Wird der Chatbot beispielsweise anstatt nach einer "Pharmacy" nach einer "Apothecary" gefragt (dieses Wort kommt nicht in den Trainingsdaten vor), kann durch die vortrainierten Word-Embeddings trotzdem eine Ähnlichkeit hergestellt werden und daher eine Klassifikation erfolgen. Bekannte Word-Embeddings sind beispielsweise [GloVe](https://nlp.stanford.edu/projects/glove/) und [Word2vec](https://radimrehurek.com/gensim/models/word2vec.html).
* **Mehr Daten sammeln:** Wie bereits im Abschnitt *Evaluation* erwähnt, könnte das hier gezeigte Modell deutlich von einer größeren Datenmenge profitieren. Dies gilt für Machine Learning-Modelle im Allgemeinen, so dass die Sammlung und Verarbeitung von qualitativ hochwertigen Daten vermutlich einen Großteil des Projektes ausmacht.

Für alle Anpassungen des Modells muss allerdings stets ein Grundsatz beachtet werden, welcher als ein **zentrales Problem des Machine Learning** verstanden werden kann: **Underfitting vs Overfitting!** Dieser [Blogpost](https://www.analyticsvidhya.com/blog/2020/02/underfitting-overfitting-best-fitting-machine-learning/) gibt einen guten, nicht-technischen Überblick über das Problem. 

Intuitiv wollen wir ein Deep Learning-Modell erhalten, welches durch das Trainingsprozedere ein *Verständnis* für die zugrundeliegenden Daten erreicht und dieses Verständnis auf neue Daten übertragen kann. Problematisch wird es, wenn das Modell: A) dieses Verständnis nie erreicht, da das Problem für das verwendete Modell zu komplex ist oder B) das Modell die Trainingsdaten lediglich *auswendig* lernt und damit die Fähigkeit zur *Generalisierung* neuer Daten verliert. Im Falle A) spricht man von *Underfitting* und bei B) handelt es sich um *Overfitting*. Die *Kapazität* des Modells, also dessen Potential mit komplexen Datenstrukturen umzugehen, muss immer im Verhältnis zu den vorliegenden Daten stehen. Die beiden Regime sollen durch folgende Abbildung verdeutlicht werden:

![Kapazität](./images/capacity_s.png "Kapazität")

In der Praxis existieren daher handfeste Strategien die Kapazität eines Modells anzupassen und damit Overfitting bzw. Underfitting zu vermeiden. Dieses Tensorflow [Tutorial](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit) gibt beispielsweise einen guten Einstieg. Wer sich näher mit der Theorie der Modellkapazität beschäftigen möchte, dem wird erneut Kapitel 5.2 von [Ian Goodfellow et al.](https://www.deeplearningbook.org/contents/ml.html) ans Herz gelegt.

### Ausblick

Im Rahmen des Projektes soll ein Deep Learning-Modell in die Praxis überführt werden. Anders ausgedrückt: das Modell soll in einem Frontend zum Einsatz kommen. Dafür müssen die Themen *Export* und *Deployment/Implementierung* geklärt werden. Eine sinnvolle Herangehensweise soll von den Teilnehmern erarbeitet werden. Bei der Verwendung von Tensorflow sollte gegebenenfalls [Tensorflow TFX](https://www.tensorflow.org/tfx) untersucht werden, wobei unzählige weitere Möglichkeiten der Bereitstellung bestehen.

Abseits von der in diesem Notebook dargestellten Aufgabe der *Textklassifikation* gibt es noch weitere Deep Learning NLP-Anwendungen. Einige der spannendsten sind:
* **Text Generation:** Wäre es nicht angebracht, wenn das Modell *intelligent* genug wäre, eine eigene Antwort zu formulieren, anstatt eine vorgegebene Antwort zufällig auszuwählen? Mit moderneren NN-Architekturen lässt sich dies umsetzen. Ein Beispielprojekt eines solchen *Conversational Chatbots* gibt es beispielsweise von [Thomas Wolf](https://medium.com/huggingface/how-to-build-a-state-of-the-art-conversational-ai-with-transfer-learning-2d818ac26313). Auf [Talk to Transformer](https://talktotransformer.com/) kann man einem solchen Netzwerk interaktiv auf den Zahn fühlen. Die Ergebnisse sind oft überraschend gut!
* **Neural Machine Translation:** Moderne Encoder-Decoder-Architekturen ermöglichen eine recht präzise, automatische Übersetzung von einer Sprache in eine andere. Viele Services (wie z.B. [DeepL](https://www.deepl.com/home)) nutzen solche Architekturen bereits. Eine Einführung in die Thematik gibt beispielsweise [Jason Brownlee](https://machinelearningmastery.com/introduction-neural-machine-translation/). Für Ansatzpunkte einer technischen Umsetzung könnte dieses [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/text/nmt_with_attention) relevant sein.
* **Text Summarization:** Text Summarization ist ein weiteres Anwendungsgebiet des NLP. Wie der Name schon vermuten lässt, geht es um die automatische Extraktion der Kerninhalte eines Textes, welche in einer *Summary* zusammengefasst werden. Dieser [Blogpost](https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/) erläutert die konzeptionellen Aspekte und gibt sogar Ansatzpunkte für eine Umsetzung in Keras/Tensorflow.